# Analysis of the results

In [ ]:
from analysis.analysis_utils import extract_all_info_from_hdf5
import numpy as np

In [ ]:
filepath = r"..\out\last\E3O2P21R12T17_S75_T0.7_M1000_N500000.h5"
adj_matrices, parameters = extract_all_info_from_hdf5(filepath)